## HW Assignment #4: 
Statistical Regression and Classification”, CRC Press, 2017: pg. 61, do 1.22 Exercises 1.22: 1-4, pg. 120: 2.14: 1-4

### Page 61 #1
In Section 1.12.1.2, the reader was reminded that the results of a cross- validation are random, due to the random partitioning into training and test sets. Try doing several runs of the linear and k-NN code in that section, comparing results.

In [1]:
# requisite function for the linear and k-NN code
xvalpart <- function(data,p) { 
    n <- nrow(data)
    ntrain <- round(p*n)
    trainidxs <- sample(1:n,ntrain ,replace=FALSE) 
        list (train=data[ trainidxs ,] , valid=data[-trainidxs ,])
}

In [2]:
# linear code
xvallm <- function(data,ycol ,predvars ,p,meanabs=TRUE){ 
    tmp <- xvalpart(data,p)
    train <- tmp$train
    valid <- tmp$valid
    # fit model to training data
    trainy <- train [ , ycol ]
    trainpreds <- train [ , predvars ]
    # using matrix form in lm() call
    trainpreds <- as.matrix(trainpreds)
    lmout <- lm(trainy ~ trainpreds)
    # apply fitted model to validation data; note
    # that %∗% works only on matrices , not data frames
    validpreds <- as.matrix(valid[,predvars]) 
    predy <- cbind(1,validpreds) * coef(lmout) ##### mine crashes on %*%, but I thought that was necessary??
    realy <- valid [ , ycol ]
    if (meanabs) return(mean(abs(predy - realy ))) 
        list (predy = predy , realy = realy )
}

In [3]:
# k-NN code
xvalknn <- function(data,ycol ,predvars ,k,p,meanabs=TRUE){
    tmp <- xvalpart(data,p)
    train <- tmp$train
    valid <- tmp$valid
    # fit model to training data
    trainy <- train [ , ycol ]
    trainpreds <- train [ , predvars ]
    # using matrix form in lm() call
    trainpreds <- as.matrix(trainpreds)
    lmout <- lm(trainy ~ trainpreds )
    # apply fitted model to validation data; note
    # that %∗% works only on matrices , not data frames
    validpreds <- as.matrix(valid[,predvars]) 
    predy <- cbind(1,validpreds) * coef(lmout)  ##### mine crashes on %*%, but I thought that was necessary??
    realy <- valid [ , ycol ]
    if (meanabs) return(mean(abs(predy - realy ))) 
        list (predy = predy , realy = realy )
}

In [4]:
# running linear 4 times
library(freqparcoord) 
data(mlb)
lin1 <- xvallm(mlb,5 ,c(4 ,6) ,2/3)
lin2 <- xvallm(mlb,5 ,c(4 ,6) ,2/3)
lin3 <- xvallm(mlb,5 ,c(4 ,6) ,2/3)
lin4 <- xvallm(mlb,5 ,c(4 ,6) ,2/3)

Loading required package: parallel
Loading required package: ggplot2
Loading required package: GGally
Loading required package: FNN
Loading required package: mvtnorm

   

   

   For a quick introduction, type ?freqparcoord, and
   run the examples, making sure to read the comments.
   

   



In [5]:
# running k-NN 4 times
library(regtools)
knn1 <- xvalknn(mlb,5 ,c(4 ,6) ,25 ,2/3)
knn2 <- xvalknn(mlb,5 ,c(4 ,6) ,25 ,2/3)
knn3 <- xvalknn(mlb,5 ,c(4 ,6) ,25 ,2/3)
knn4 <- xvalknn(mlb,5 ,c(4 ,6) ,25 ,2/3)

Loading required package: dummies
dummies-1.5.6 provided by Decision Patterns

Loading required package: car


In [6]:
# make a table out of the different runs to show how they differ due to random partitioning
lin_array <- c(lin1,lin2,lin3,lin4)
knn_array <- c(knn1,knn2,knn3,knn4)
runs_matrix <- matrix(c(lin_array,knn_array),ncol = 4)
colnames(runs_matrix) <- c("run 1", "run 2", "run 3", "run 4")
rownames(runs_matrix) <- c("Linear", "k-NN")
runs_matrix

,run 1,run 2,run 3,run 4
Linear,2260.419,2323.918,2365.854,2425.644
k-NN,2153.631,2128.513,2369.865,2420.348







### Page 61 #2
Extend (1.28) to include interaction terms for age and gender, and age2 and gender. Run the new model, and find the estimated effect of being female, for a 32-year-old person with a Master’s degree.

In [9]:
# lets start with our standard model, with no interaction terms (besides age^2)
library(freqparcoord)
data(prgeng)
prgeng$age2 <- prgeng$age^2
edu <- prgeng$educ
prgeng$ms <- as.integer(edu == 14) 
prgeng$phd <- as.integer(edu == 16)
prgeng$fem <- prgeng$sex - 1
tmp <- prgeng[edu >= 13,]
pe <- tmp[ ,c(1 ,12 ,9 ,13 ,14 ,15 ,8)]
pe <- as.matrix(pe)

In [10]:
# run the first model
lm(wageinc ~
    age+age2+wkswrkd+ms+phd+fem , data=prgeng )


Call:
lm(formula = wageinc ~ age + age2 + wkswrkd + ms + phd + fem, 
    data = prgeng)

Coefficients:
(Intercept)          age         age2      wkswrkd           ms          phd  
  -81136.70      3900.35       -40.33      1196.39     15431.07     23183.97  
        fem  
  -11484.49  


In [11]:
# but now lets add the interaction terms.
# age * gender
# age^2 * gender

prgeng$agefem <- prgeng$age * prgeng$fem
prgeng$age2fem <- prgeng$age2 * prgeng$fem 

prob2 <- lm(wageinc ~
age+age2+wkswrkd+ms+phd+fem+agefem+age2fem ,
data=prgeng)
summary(prob2)


Call:
lm(formula = wageinc ~ age + age2 + wkswrkd + ms + phd + fem + 
    agefem + age2fem, data = prgeng)

Residuals:
   Min     1Q Median     3Q    Max 
-99246 -20309  -4191  12751 292704 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -90553.725   3690.091 -24.540  < 2e-16 ***
age           4349.241    187.037  23.253  < 2e-16 ***
age2           -45.173      2.148 -21.035  < 2e-16 ***
wkswrkd       1194.291     21.884  54.574  < 2e-16 ***
ms           15364.219    738.887  20.794  < 2e-16 ***
phd          23017.206   1625.999  14.156  < 2e-16 ***
fem          33996.574   8219.263   4.136 3.55e-05 ***
agefem       -2164.553    402.202  -5.382 7.46e-08 ***
age2fem         23.949      4.737   5.056 4.32e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 42620 on 20081 degrees of freedom
Multiple R-squared:  0.2368,	Adjusted R-squared:  0.2365 
F-statistic: 778.7 on 8 and 20081 DF,  p-value: < 2.2e-16


In [14]:
# find the estimated effect of being female, for a 32-year-old person with a Master’s degree. 
# Assume she works every day of the year

prob2_woman <- data.frame(age=32,age2=32^2,wkswrkd=52,ms=1,phd=0,fem=1,agefem=32*1,age2fem=(32^2)*1)

predict(prob2,prob2_woman,interval="predict")

,fit,lwr,upr
1,69086.59,-14477.21,152650.4


### Page 61 #3
Consider the bodyfat data mentioned in Section 1.2. Use lm() to form a prediction equation for density from the other variables (skipping the first three), and comment on whether use of indirect methods in this way seems feasible.

In [16]:
library(mfp)
data(bodyfat)

Loading required package: survival


In [17]:
?bodyfat

In [19]:
out <- lm(density ~ age + weight + height + neck + chest + abdomen + hip + thigh + knee + ankle + forearm + wrist, data = bodyfat)

In [20]:
summary(out)


Call:
lm(formula = density ~ age + weight + height + neck + chest + 
    abdomen + hip + thigh + knee + ankle + forearm + wrist, data = bodyfat)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.022194 -0.007191  0.000507  0.006507  0.035790 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.135e+00  4.030e-02  28.150  < 2e-16 ***
age         -1.282e-04  7.513e-05  -1.707  0.08916 .  
weight       2.077e-04  1.227e-04   1.693  0.09169 .  
height       1.657e-04  2.233e-04   0.742  0.45863    
neck         1.011e-03  5.395e-04   1.875  0.06208 .  
chest        9.640e-05  2.301e-04   0.419  0.67566    
abdomen     -2.242e-03  1.999e-04 -11.217  < 2e-16 ***
hip          5.901e-04  3.387e-04   1.742  0.08273 .  
thigh       -7.452e-04  3.238e-04  -2.301  0.02223 *  
knee         1.372e-05  5.618e-04   0.024  0.98054    
ankle       -6.044e-04  5.154e-04  -1.173  0.24204    
forearm     -1.215e-03  4.430e-04  -2.742  0.00657 ** 
wrist     

Based on this linear model, it appears that there are only a handful of significant variables. One of those variables is the abdomen, which intuitively makes sense. The measures that are significant: abdomen, thigh, forearm, and wrist. are easily measured. additionally, this model can easily be refined from it's current state, and improve the adjusted R-squared, which is already 0.7225

### Page 61 #4
In Section 1.19.5.2, we gave this intuitive explanation:

In other words, the national mean height is a weighted average of the state means, with the weight for each state being its proportion of the national population. Replace state by gender in the following.

(a) Write English prose that relates the overall mean height of people and the gender-specific mean heights.

(b) Write English prose that relates the overall proportion of people taller than 70 inches to the gender-specific proportions.

#### (a) Write English prose that relates the overall mean height of people and the gender-specific mean heights.

"While the overall mean height of people might be something around 5 foot 8 inches, the gender specific mean heights are different. However, you can use a weighted mean to arrive at the overall mean height, if all you have is the gender specific mean heights, and the number of people of each gender. You simply weight the female average by the number of females and the male average by the number of males, and then average the two."

#### (b) Write English prose that relates the overall proportion of people taller than 70 inches to the gender specific proportions.

"Imagine you were to draw a line down the population of humans, and on one side you had people taller than 70 inches, and on the other you had people shorter than 70 inches. If you then separated each of these groups based on male/female, then you would have two proportions: % Males over 70 inches, and % Females over 70 inches. These two proportions would undoubtedly be different from the overall population proportion. However, if you weight the proportion of males over 70 by the number of males in the population, and do the same for females, then average the two resulting proportions, you'll arrive at the population proportion.

### Page 120 #1
Consider the census data in Section 1.16.1.

(a) Form an approximate 95% confidence interval for β6 in the model (1.28).

(b) Form an approximate 95% confidence interval for the gender effect for Master’s degree holders, β6 + β7, in the model (1.28).

In [21]:
# This is the data and the model where the beta-6 being referred to come from
library(freqparcoord)
data(prgeng)
prgeng$age2 <- prgeng$age^2
edu <- prgeng$educ
prgeng$ms <- as.integer(edu == 14) 
prgeng$phd <- as.integer(edu == 16) 
prgeng$fem <- prgeng$sex - 1
tmp <- prgeng[edu >= 13,]
pe <- tmp[ ,c(1 ,12 ,9 ,13 ,14 ,15 ,8)]
pe <- as.matrix(pe)

In [22]:
lin_cen <- lm(wageinc ~
    age+age2+wkswrkd+ms+phd+fem , data=prgeng )
summary(lin_cen)


Call:
lm(formula = wageinc ~ age + age2 + wkswrkd + ms + phd + fem, 
    data = prgeng)

Residuals:
   Min     1Q Median     3Q    Max 
-98563 -20332  -4273  12781 290808 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -81136.70    3284.75  -24.70   <2e-16 ***
age           3900.35     168.77   23.11   <2e-16 ***
age2           -40.33       1.95  -20.68   <2e-16 ***
wkswrkd       1196.39      21.89   54.65   <2e-16 ***
ms           15431.07     738.80   20.89   <2e-16 ***
phd          23183.97    1626.70   14.25   <2e-16 ***
fem         -11484.49     705.30  -16.28   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 42650 on 20083 degrees of freedom
Multiple R-squared:  0.2356,	Adjusted R-squared:  0.2354 
F-statistic:  1032 on 6 and 20083 DF,  p-value: < 2.2e-16


#### (a) Form an approximate 95% confidence interval for β6 in the model (1.28).


In [23]:
std_err <- 705.3
fem_est <- -11484.49
lower <- fem_est - (2*std_err)
upper <- fem_est + (2*std_err)

In [24]:
paste("For the 95% confidence interval on the effect of being female, the lower bound is ", lower, ". And the upper bound is ", upper)

[1] "For the 95% confidence interval on the effect of being female, the lower bound is  -12895.09 . And the upper bound is  -10073.89"

#### (b) Form an approximate 95% confidence interval for the gender effect for Master’s degree holders, β6 + β7, in the model (1.28).

In [114]:
prgeng$msfem <- prgeng$ms * prgeng$fem
prgeng$phdfem <- prgeng$phd * prgeng$fem
lin_b <- lm(wageinc ~
age+age2+wkswrkd+ms+phd+fem+msfem+phdfem ,
data=prgeng)
summary(lin_b)


Call:
lm(formula = wageinc ~ age + age2 + wkswrkd + ms + phd + fem + 
    msfem + phdfem, data = prgeng)

Residuals:
    Min      1Q  Median      3Q     Max 
-100361  -20374   -4247   12824  289893 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -81216.778   3283.880 -24.732  < 2e-16 ***
age           3894.320    168.735  23.080  < 2e-16 ***
age2           -40.293      1.949 -20.669  < 2e-16 ***
wkswrkd       1195.309     21.889  54.609  < 2e-16 ***
ms           16433.668    846.655  19.410  < 2e-16 ***
phd          25325.315   1759.281  14.395  < 2e-16 ***
fem         -10276.797    804.498 -12.774  < 2e-16 ***
msfem        -4157.253   1728.329  -2.405  0.01617 *  
phdfem      -14061.635   4605.664  -3.053  0.00227 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 42640 on 20081 degrees of freedom
Multiple R-squared:  0.2361,	Adjusted R-squared:  0.2358 
F-statistic: 775.9 on 8 and 20081 DF,  p-value: < 

In [116]:
fem_std_err <- 804.498
msfem_std_err <- 1728.329
fem_est <- -10276.797
msfem_est <- -4157.253

# I don't know what this one is asking :/

### Page 120 #2
The full bikeshare dataset spans 3 years’ time. Our analyses here have only used the first year. Extend the analysis in Section 2.8.5 to the full data set, adding dummy variables indicating the second and third year. Form an approximate 95% confidence interval for the difference between the coefficients of these two dummies.


In [25]:
# download the data
bikes <- read.csv("day.csv", header = TRUE)

In [26]:
# Make the dummy variables and the variables that we made in the book earlier on: temp2 and clearday
library(dummies)

bikes <- cbind(bikes, dummy(bikes$yr, sep = "YearDummy_"))

bikes$temp2 <- bikes$temp^2

bikes$clearday <- as.integer(bikes$weathersit == 1)

There are only 2 years :/

In [138]:
bikes

instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,NAYearDummy_0,NAYearDummy_1,temp2,clearday
1,2011-01-01,1,0,1,0,6,0,2,0.3441670,0.3636250,0.805833,0.1604460,331,654,985,1,0,0.118450924,0
2,2011-01-02,1,0,1,0,0,0,2,0.3634780,0.3537390,0.696087,0.2485390,131,670,801,1,0,0.132116256,0
3,2011-01-03,1,0,1,0,1,1,1,0.1963640,0.1894050,0.437273,0.2483090,120,1229,1349,1,0,0.038558820,1
4,2011-01-04,1,0,1,0,2,1,1,0.2000000,0.2121220,0.590435,0.1602960,108,1454,1562,1,0,0.040000000,1
5,2011-01-05,1,0,1,0,3,1,1,0.2269570,0.2292700,0.436957,0.1869000,82,1518,1600,1,0,0.051509480,1
6,2011-01-06,1,0,1,0,4,1,1,0.2043480,0.2332090,0.518261,0.0895652,88,1518,1606,1,0,0.041758105,1
7,2011-01-07,1,0,1,0,5,1,2,0.1965220,0.2088390,0.498696,0.1687260,148,1362,1510,1,0,0.038620896,0
8,2011-01-08,1,0,1,0,6,0,2,0.1650000,0.1622540,0.535833,0.2668040,68,891,959,1,0,0.027225000,0
9,2011-01-09,1,0,1,0,0,0,1,0.1383330,0.1161750,0.434167,0.3619500,54,768,822,1,0,0.019136019,1
10,2011-01-10,1,0,1,0,1,1,1,0.1508330,0.1508880,0.482917,0.2232670,41,1280,1321,1,0,0.022750594,1


In [139]:
# start with the original bikeshare data
lmout <- lm(registered ~ temp+temp2+workingday+clearday+NAYearDummy_0+NAYearDummy_1, data = bikes)
summary(lmout)


Call:
lm(formula = registered ~ temp + temp2 + workingday + clearday + 
    NAYearDummy_0 + NAYearDummy_1, data = bikes)

Residuals:
    Min      1Q  Median      3Q     Max 
-4503.3  -463.8    14.0   481.2  1902.4 

Coefficients: (1 not defined because of singularities)
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -737.34     219.05  -3.366 0.000803 ***
temp           14449.21     906.77  15.935  < 2e-16 ***
temp2         -10590.58     912.67 -11.604  < 2e-16 ***
workingday       953.34      60.91  15.651  < 2e-16 ***
clearday         621.81      59.72  10.413  < 2e-16 ***
NAYearDummy_0  -1716.25      56.68 -30.278  < 2e-16 ***
NAYearDummy_1        NA         NA      NA       NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 762.7 on 725 degrees of freedom
Multiple R-squared:  0.7627,	Adjusted R-squared:  0.761 
F-statistic:   466 on 5 and 725 DF,  p-value: < 2.2e-16


One of the dummy variables for year was actually not defined because of a singularity. This is likely due to some issue with the data, given that it has changed since it was referenced in this book.

### Page 120 #3
Suppose we are studying growth patterns in children, at k particular ages. Denote the height of the ith child in our sample data at age j by Hij, with Hi = (Hi1,...,Hik)′ denoting the data for child i. Suppose the population distribution of each Hi is k-variate normal with mean vector μ and covariance matrix Σ. Say we are interested in successive differences in heights,Dij =Hi,j+1−Hij, j=1,2,...,k−1. DefineDi =(Di1,...,Dik)′. Explain why each Di is (k−1)-variate normal, and derive matrix expressions for the mean vector and covariance matrices.

Honestly I don't know what this question is asking. While I might be able to look at another student's answer, I would essentially just be copying it at this point.

### Page 120 #4
In the simulation in Section 2.9.3, it is claimed that ρ2 = 0.50. Confirm this through derivation.

Honestly I don't know what this question is asking. While I might be able to look at another student's answer, I would essentially just be copying it at this point.